# Strategy Serialization with BoFire

## Imports

In [54]:
import json
from pydantic import parse_obj_as


from bofire.data_models.domain.api import Inputs, Outputs, Domain
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.multi import DTLZ2
from bofire.data_models.strategies.api import SoboStrategy as SoboStrategyDataModel
from bofire.data_models.strategies.api import QnehviStrategy as QnehviStrategyDataModel
from bofire.data_models.strategies.api import RandomStrategy as RandomStrategyDataModel
from bofire.data_models.strategies.api import AnyStrategy
from bofire.data_models.acquisition_functions.api import qNEI
import bofire.strategies.api as stategies
from bofire.data_models.surrogates.api import BotorchSurrogates, AnySurrogate, SingleTaskGPSurrogate
from bofire.data_models.kernels.api import ScaleKernel, RBFKernel




## Single Objective Problem Setup

In [2]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=10)
experiments = benchmark.f(samples, return_complete=True)

## Random Strategy

The random strategy and other strategies that just inherit from `Strategy` and not `PredictiveStrategy` are special as they do not need defined output features in the domain and they do not need a call to `tell` before the `ask`. Furthermore they online provide input features in the candidates and no predictions for output features.

In [20]:
# setup the data model
domain = Domain(input_features=benchmark.domain.input_features)
strategy_data = RandomStrategyDataModel(domain=domain)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "RandomStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "output_features": {"type": "Outputs", "features": []}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 256}'

In [21]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=5)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=-1.2325343376159328), 'x_2': InputValue(value=-3.1869720129999273)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=2.8172878246860407), 'x_2': InputValue(value=3.3265172127152063)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=1.8054911073833129), 'x_2': InputValue(value=-2.9451929437162274)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-1.1907462496383197), 'x_2': InputValue(value=-2.966009463557924)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=0.2076155277831946), 'x_2': InputValue(value=-3.9212687100549504)}, outputValues=None)]

## SOBO Strategy

This will fail as SOBO is a predictive strategy which needs also output feature definitions, which is missing in the domain from before.

In [22]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

ValidationError: 1 validation error for SoboStrategy
domain
  no output feature specified (type=value_error)

Next try with a correct domain:

In [42]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=benchmark.domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "SoboStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "output_features": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MaximizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 742, "num_sobol_samples": 512, "num_restarts": 8, "num_raw_samples": 1024, "descriptor_method": "EXHAUSTIVE", "categorical_method": "EXHAUSTIVE", "discrete_method": "EXHAUSTIVE", "surrogate_specs": {"surrogates": [{"type": "SingleTaskGPSurrogate", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "ou

This will fail as SOBO is a predictive strategy which means we have to provide training data before:

In [43]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=2)

ValueError: Not enough experiments available to execute the strategy.

which is done by using the `tell` method:

In [44]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=2)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=-0.2588482202321763), 'x_2': InputValue(value=-4.0)}, outputValues={'y': OutputValue(predictedValue=181.01282121894124, standardDeviation=24.64999493071629, objective=181.01282121894124)}),
 Candidate(inputValues={'x_1': InputValue(value=0.20055265244293544), 'x_2': InputValue(value=-2.2730259435818407)}, outputValues={'y': OutputValue(predictedValue=183.32953981882224, standardDeviation=22.250343695582654, objective=183.32953981882224)})]

We can also save the trained models of the strategy, for more info look at the `model_serial.ipynb` notebook. It could be that the `dumps` command fails here. But this is already fixed in the main branch of the `linear_operator` package, and if not yet, it should be available in main soon.

In [46]:
jsurrogate_spec = strategy_data.surrogate_specs.surrogates[0].json()
dump = strategy.surrogate_specs.surrogates[0].dumps()

## MOBO Strategy

As example for a multiobjective strategy we are using here the Qnehvi stratey. Related strategies would be Qparego, MultiplicativeSobo etc. To use it, we have to first generate a multiobjective domain.

In [51]:
benchmark = DTLZ2(dim=6)
samples = benchmark.domain.inputs.sample(n=10)
experiments = benchmark.f(samples, return_complete=True)

Now the strategy spec is setup. Note that we can define there exactly which model to use.

In [58]:
# setup the data model
strategy_data = QnehviStrategyDataModel(
    domain=benchmark.domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            SingleTaskGPSurrogate(
                input_features=benchmark.domain.input_features,
                output_features=Outputs(features=[benchmark.domain.outputs[0]]),
                kernel=ScaleKernel(base_kernel=RBFKernel(ard=False))
            )
        ]
    )
)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "QnehviStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_0", "lower_bound": 0.0, "upper_bound": 1.0}, {"type": "ContinuousInput", "key": "x_1", "lower_bound": 0.0, "upper_bound": 1.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": 0.0, "upper_bound": 1.0}, {"type": "ContinuousInput", "key": "x_3", "lower_bound": 0.0, "upper_bound": 1.0}, {"type": "ContinuousInput", "key": "x_4", "lower_bound": 0.0, "upper_bound": 1.0}, {"type": "ContinuousInput", "key": "x_5", "lower_bound": 0.0, "upper_bound": 1.0}]}, "output_features": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "f_0", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}}, {"type": "ContinuousOutput", "key": "f_1", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 649

Generate the candidates.

In [59]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=1)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_0': InputValue(value=0.34451258412632535), 'x_1': InputValue(value=0.2021898805014269), 'x_2': InputValue(value=0.9130968117758534), 'x_3': InputValue(value=0.3012002108665261), 'x_4': InputValue(value=0.0), 'x_5': InputValue(value=0.0)}, outputValues={'f_0': OutputValue(predictedValue=0.6777183521800488, standardDeviation=0.2366029815403661, objective=-0.6777183521800488), 'f_1': OutputValue(predictedValue=0.877560768553107, standardDeviation=0.532737213995751, objective=-0.877560768553107)})]

Again the models can be saved. Note that we have two models here as we have two features in `domain.output_features`.

In [62]:
jsurrogate_specs = [surrogate.json() for surrogate in strategy_data.surrogate_specs.surrogates]
dumps = [surrogate.dumps() for surrogate in strategy.surrogate_specs.surrogates]

This is the general setup of how it should work. What is missing and still needs to be implemented into bofire is to sideload already fitted models which is important for deterministic ones.